In [1]:
from __future__ import print_function, division
from io import open
import string
import re
import random
import pandas as pd
import os
import sys
import argparse
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score  
from sklearn.metrics import roc_curve 

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

try:
    import cPickle as pickle
except:
    import pickle

In [10]:
dc_l=[15566,109,84,267,62,144,379,149,61,231,13202,154,345,83,63,85,4263,184,1036]
dc_a_l=[109,267,144,379,149,61,231,13202,345,83,1036]
dc_g_r_l=[15566,84,62,154,63,85,4263,184]

for h in [143]: #[104 ,118 ,131 ,154 ,212 ,225 ,384 ,85 ,14 ,143]:
    t_file= '/data/projects/cerner_dev/cl2_hosp_pdata/h50_cl2_all_hosp_'+ str(h)
    t_seqfile= t_file +'.visits.'
    t_labfile= t_file +'.labels.'
    t_timefile= t_file +'.days.'
    t_ptfile= t_file +'.pts.'
    
    for s in ['test','train','valid']:
        fset=[]
        t_seqfile_s=t_seqfile+s
        t_labfile_s=t_labfile+s
        t_timefile_s= t_timefile +s
        t_ptfile_s= t_ptfile +s
        set_x = pickle.load(open(t_seqfile_s, 'rb'),encoding= 'bytes')
        set_y = pickle.load(open(t_labfile_s, 'rb'),encoding= 'bytes')
        set_t = pickle.load(open(t_timefile_s, 'rb'),encoding= 'bytes')
        set_p = pickle.load(open(t_ptfile_s, 'rb'),encoding= 'bytes')
        for pt_idx in range(len(set_y)):
            pt_sk= set_p[pt_idx]
            pt_lbl= set_y[pt_idx]
            pt_vis= set_x[pt_idx]
            pt_td= set_t[pt_idx]
            d_gr=[]
            n_seq=[]
            d_a_v=[]
            for v in range(len(pt_vis)):
                    nv=[]
                    nv.append([pt_td[v]])
                    for c in pt_vis[v]:
                        if (c in dc_g_r_l) and (c not in d_gr): 
                                d_gr.append(c) 
                        elif c in dc_a_l:
                                d_a_v.append(c)                  
                        else: 
                            nv.append(c)                   
                   #n_seq.append([d_a_v,nv]) ### age group is a visit sublist
                    n_seq.append(nv)
            d_gr.append(d_a_v[0]) 
            #n_pt=[pt[0],d_gr ,n_seq]
            n_pt= [pt_sk,pt_lbl,d_gr ,n_seq]
            fset.append(n_pt)
        
        print (s,len(fset))        
              
        fname='/data/projects/py_ehr_2/Data/h_'+str(h)+'cl2_combined_tp.'+s
        pickle.dump(fset, open(fname, 'wb'), -1)

test 8956
train 29504
valid 4269


In [2]:
for s in ['test','train','valid']:    
    fname='/data/projects/py_ehr_2/Data/h_143cl2_combined_tp.'+s
    fset1=pickle.load(open(fname, 'rb'),encoding='bytes')
    print (s,len(fset1),fset1[0])

test 8956 [310327, 1, [63, 62, 61], [[[0], 1456], [[9], 124, 63, 62, 375, 239], [[32], 239, 62, 63], [[29], 239, 62, 63], [[189], 722, 63, 62, 129, 371, 399, 230], [[16], 62, 63, 230], [[98], 62, 239, 63, 449], [[28], 793, 230, 62, 124, 63, 239, 449], [[18], 239, 63, 62], [[28], 239, 62, 63], [[35], 239, 63, 62], [[28], 239, 62, 63], [[28], 63, 239, 62], [[28], 62, 239, 63], [[17], 230, 449, 59, 945, 124, 62, 63], [[12], 62, 63, 239], [[28], 63, 239, 62]]]
train 29504 [310048, 1, [63, 84, 231], [[[0], 1416], [[15], 57, 457, 84, 63], [[13], 63, 1416, 2504, 4515, 84, 925], [[21], 518, 57, 232, 84, 150, 63], [[43], 457, 132, 556, 57, 84, 63], [[8], 2669, 8, 556, 2375, 84, 384, 421, 1788, 63, 6], [[40], 68, 63, 1416, 84], [[15], 449, 57, 84, 457, 556, 63], [[28], 84, 559, 63, 675, 57], [[48], 1416, 63, 84], [[15], 388, 457, 57, 63, 84, 449], [[5], 63, 84, 268], [[19], 195, 457, 343, 1182, 53, 173, 553, 207, 84, 751, 34, 12, 31, 409, 212, 16, 158, 22, 80, 1259, 6, 135, 956, 189, 657, 56, 47

In [8]:
emb_types = pickle.load(open('/data/projects/cerner_dev/cl2_hosp_pdata/h50_cl2_all.types.train', 'rb'),encoding='bytes')

for x in emb_types:
    if ('a_' in str(x)) or ( 'g_' in str(x))or ('r_' in str(x)) : print (x,emb_types[x])

b'r_White' 85
b'a_75-79 years' 109
b'a_35-39 years' 13202
b'a_45-49 years' 379
b'g_Male' 62
b'a_40-44 years' 1036
b'a_65-69 years' 149
b'a_50-54 years' 83
b'r_MultiRacial' 4263
b'r_AsianPacific' 154
b'g_Female' 84
b'r_AfricanAmerican' 63
b'a_55-59 years' 345
b'a_70-74 years' 61
b'a_80-84 years' 231
b'g_UnIdentified' 15566
b'a_60-64 years' 144
b'a_85+ years' 267
b'r_UnIdentified' 184


In [ ]:
'''
pickle.dump(train_sl, open('hf50_bal_train', 'wb'), -1)
pickle.dump(test_sl, open('hf50_bal_test', 'wb'), -1)
pickle.dump(valid_sl, open('hf50_bal_valid', 'wb'), -1)